In [1]:
import gym
from room.envs import GymEnvWrapper
from typing import List, Optional, Union, Dict
from room.common.typing import CfgType
import torch
from tqdm import trange

from room import notice
from room.agents import Agent, DQN
from room.common.callbacks import Callback
from room.common.utils import get_param
from room.envs.wrappers import EnvWrapper
from room.loggers import Logger
from room.memories import Memory, RandomMemory
from room.trainers.base import Trainer
from room.agents.policies import Policy
from room.networks.blocks import MLP
import time

In [2]:
env = gym.make("CartPole-v1")
env = GymEnvWrapper(env)

In [3]:
agent = DQN(policy = [], optimizer=torch.optim.Adam, device="cuda", cfg={}, epsilon=0.1, gamma=0.01, lr=1e-3)

[WARNING] No policy is not defined in the config file.
[WARNING] No epsilon is not defined in the config file.
[WARNING] No gamma is not defined in the config file.
Sequential(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=2, bias=True)
  (relu2): ReLU()
)
[WARNING] No lr is not defined in the config file.
[INFO] Loading optimizer same as the aliases.


In [4]:
memory = RandomMemory(capacity=10)

In [5]:
state = env.reset()
state = state[0]

In [6]:
for t in trange(1000):
    with torch.no_grad():

        action = agent.act(state[0])

    next_state, reward, terminated, truncated, info = env.step(action)
    memory.add(
        {
            "state": state,
            "action": action,
            "reward": reward,
            "next_state": next_state,
            "terminated": terminated,
            "truncated": truncated,
            "info": info,
        }
    )

    if not memory.is_full():
        continue
    else:
        batch = memory.sample(batch_size=3)
        agent.learn(batch)

    with torch.no_grad():
        if terminated.any() or truncated.any():
            states, infos = env.reset()

  0%|          | 1/1000 [00:00<09:45,  1.71it/s]/home/ksterx/Applications/miniconda3/envs/ai/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(
100%|██████████| 1000/1000 [00:01<00:00, 640.16it/s]


In [7]:
agent.play(env)